### Integrating with Apache Airflow
**Description**: Integrate Great Expectations with Apache Airflow to run data quality checks automatically in your DAG.

**Steps**:
1. Install Airflow (if you haven't already):
2. Airflow DAG Integration:
    - Create a DAG file:
3. Deploy and Test:
    - Place this file in your Airflow DAGs directory and start your Airflow scheduler.
    - Open the Airflow UI and trigger the DAG to see it run your expectations.

In [ ]:
# Write your code from here
from airflow import DAG
from airflow.operators.python import PythonOperator
from datetime import datetime, timedelta
import great_expectations as ge

def run_great_expectations():
    try:
        context = ge.get_context()
        # Name of your expectation suite
        suite_name = "my_expectation_suite"
        # Path to your data file to validate (update as needed)
        data_path = "/path/to/your/data.csv"
        
        # Load data as a GE dataset
        batch = context.get_batch({
            "datasource": "my_datasource",
            "data_connector_name": "default_inferred_data_connector_name",
            "data_asset_name": data_path,
        }, suite_name)
        
        results = context.run_validation_operator(
            "action_list_operator",
            assets_to_validate=[batch],
            run_name=f"validation_{datetime.now().isoformat()}"
        )
        if not results["success"]:
            raise Exception("Data Quality Check Failed")
    except Exception as e:
        raise RuntimeError(f"Great Expectations validation failed: {e}")

default_args = {
    "owner": "airflow",
    "depends_on_past": False,
    "start_date": datetime(2025, 1, 1),
    "retries": 1,
    "retry_delay": timedelta(minutes=5),
}

with DAG(
    dag_id="great_expectations_data_quality_check",
    default_args=default_args,
    schedule_interval="@daily",
    catchup=False,
    tags=["data_quality"],
) as dag:

    run_validation = PythonOperator(
        task_id="run_ge_validation",
        python_callable=run_great_expectations,
    )
